<a href="https://colab.research.google.com/github/BG4Finance/On_Going_Proj/blob/master/GoldenCross_Alg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip3 install alpaca-trade-api

In [0]:
import alpaca_trade_api as alp
import numpy as np
import pandas as pd
import statistics
import sys
import time
import datetime as dt
from datetime import datetime, timedelta
from pytz import timezone

In [0]:
api = alp.REST(key_id = 'PKC3Q62740ML0K30TPVC', secret_key = 'TWyyiQcOqpRKyv2xNOt1XLAKZPW8vqvGraHtneMB', base_url="https://paper-api.alpaca.markets")
api.list_positions()
api_time_format = '%Y-%m-%dT%H:%M:%S.%f-04:00'
account = api.get_account()
account.status

'ACTIVE'

In [0]:
today = (dt.datetime.now())
print(today)
equity = float(account.equity)
margin_multiplier = float(account.multiplier)
total_buying_power = margin_multiplier * equity
print(f'Initial total buying power = {total_buying_power}, {total_buying_power*100/equity}%')

2019-12-02 08:48:21.614397
Initial total buying power = 400000.0, 400.0%


In [0]:
now = (dt.datetime.now()).date().strftime(api_time_format)
data = api.get_barset('TSLA',timeframe='day', limit=80, end=now)
sma6 = 0
sma4 = 0
for i in range(1,60):
  sma6 = (sma6 + data['TSLA'][-i].c)
sma6=round(sma6/60,2)

for i in range(1,40):
  sma4 = (sma4 + data['TSLA'][-i].c)
sma4=round(sma4/40,2)

print(f'Simple moving average, 60D:{sma6} 40D:{sma4}')
if sma4 > sma6 :
  print('BUY PHASE')
elif sma6 > sma4:
  print('SELL PHASE')
else:
  print('GOLD CROSS IN COURSE')

Simple moving average, 60D:278.9 40D:298.48
BUY PHASE


In [0]:
class GX:
  def __init__(self):
    self.alpaca=alp.REST(key_id = 'PKC3Q62740ML0K30TPVC', secret_key = 'TWyyiQcOqpRKyv2xNOt1XLAKZPW8vqvGraHtneMB', base_url="https://paper-api.alpaca.markets")
    stockUniverse = ['DOMO', 'TLRY', 'SQ', 'MRO', 'AAPL', 'GM', 'SNAP', 'SHOP', 'SPLK', 'BA', 'AMZN', 'SUI', 'SUN', 'TSLA', 'CGC', 'SPWR', 'NIO', 'CAT', 'MSFT', 'PANW', 'OKTA', 'TWTR', 'TM', 'RTN', 'ATVI', 'GS', 'BAC', 'MS', 'TWLO', 'QCOM', ]
    self.allStocks = []

    for stock in stockUniverse:
      self.allStocks.append([stock, 0])

    self.long = []
    self.short = []
    self.qShort = None
    self.qLong = None
    self.adjustedQLong = None
    self.adjustedQShort = None
    self.blacklist = set()
    self.longAmount = 0
    self.shortAmount = 0
    self.timeToClose = None

  def run(self):
    orders = self.alpaca.list_orders(status="open")
    for order in orders:
      self.alpaca.cancel_order(order.id)

    print("Waiting for market to open...")
    tAMO = threading.Thread(target=self.awaitMarketOpen)
    tAMO.start()
    tAMO.join()
    print("Market opened.")

    while True:
      clock = self.alpaca.get_clock()
      closingTime = clock.next_close.replace(tzinfo=datetime.timezone.utc).timestamp()
      currTime = clock.timestamp.replace(tzinfo=datetime.timezone.utc).timestamp()
      self.timeToClose = closingTime - currTime

      if(self.timeToClose < (60 * 15)): # Close all positions when 15 minutes til market close.
        print("Market closing soon.  Closing positions.")
        positions = self.alpaca.list_positions()

        for position in positions:
          if(position.side == 'long'):
            orderSide = 'sell'
          else:
            orderSide = 'buy'
          qty = abs(int(float(position.qty)))
          respSO = []
          tSubmitOrder = threading.Thread(target=self.submitOrder(qty, position.symbol, orderSide, respSO))
          tSubmitOrder.start()
          tSubmitOrder.join()
        print("Sleeping until market close (15 minutes).")
        time.sleep(60 * 15)

        

In [0]:
def get_ratings(symbols, algo_time):
  assets = api.list_assets()
  assets = [asset for asset in assets if asset.tradable ]
  ratings = pd.DataFrame(columns=['symbol', 'rating', 'price'])
  index = 0
  batch_size = 200 # The maximum number of stocks to request data for
  window_size = 80 # The number of days of data to consider
  formatted_time = None
  if algo_time is not None:
    # Convert the time to something compatable with the Alpaca API.
    formatted_time = algo_time.date().strftime(api_time_format)
    print(formatted_time)
  while index < len(assets):
    symbol_batch = [asset.symbol for asset in assets[index:index+batch_size]]
    barset = api.get_barset(
        symbols=symbol_batch,
        timeframe='day',
        limit=window_size,
        end=formatted_time
        )
    for symbol in symbol_batch:
      bars = barset[symbol]
      price = bars[-1].c
      print(price)
      if len(bars) == window_size:
        # Make sure we aren't missing the most recent data.
        latest_bar = bars[-1].t.to_pydatetime().astimezone(timezone('EST'))
        gap_from_present = algo_time - latest_bar
        if gap_from_present.days > 1:
          continue
        
        price = bars[-1].c
        print(price)

In [0]:
today = datetime.utcnow()
get_ratings('FB',today)

In [0]:
# Daily OHLCV dataframe
aapl_daily = api.polygon.historic_agg('day', 'AAPL', limit=1000).df
# Dictionary of most recent earning stats for each company
list_earning = api.polygon.earnings(['MSFT','FB','AMZN'])
# Returns a list articles and their meta-data
tsla_news = api.polygon.news('TSLA')

In [0]:
# Lists currently open trades
positions = api.list_positions()
# Places a limit order
api.submit_order('AAPL',10,'buy','limit','gtc',170.50,)
# Lists all open orders
orders = api.list_orders()